# - Applying Back-propogation algorithm without major libraries

In [6]:
#Importing neccasary libraries
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tqdm import tqdm 

# Defining an activation fucntions for the second layer - sigmoid activation function 
def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s

# Defining an activation function for the final layer : Softmax activation function since there are 4 classes to be determined  
def softmax(z):
    expz = np.exp(z - z.max())
    return expz / np.sum(expz, axis=0)


# Importing the dataset required to be trained for the model
x = pd.read_csv('Input_data.csv')
y = pd.read_csv('labels_Input_data.csv')
y = y.astype(np.int)

#Performing a 80-20% split of the provided dataset, into training and testinf data respectively.
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

# Defining a class BPNN to incoroporate all the neccasary functions for model development
class BPNN(object):

    #Initializing the model parameters
    def __init__(self, hidden_size, epochs=30, lr=0.01):
        self.epochs = epochs #Number of times the model gets updated
        self.lr = lr #learning rate
        self.input_size = 784 #Input_size : the features of the dataset as the input to this model
        self.output_size = 4 #output_size : the 4 different classes that are the output of the question
        self.hidden = hidden_size #number of nodes in the hidden layer, this is uder dependant
        
        print('The designed network will have ' +str(hidden_size)+ ' number of nodes in the hidden layer')
        print('-----')
        
        self.weight1 = np.random.normal(0, 0.2, [self.input_size, self.hidden]) # random weight initialization between the input layer and the hidden layer
        self.bias1 = np.zeros((1, self.hidden)) # Adding bias to the first layer of weights
        self.weight2 = np.random.normal(0, 0.2, [self.hidden, self.output_size]) #random weight initialization between the hidden layer and the output layer
        self.bias2 = np.zeros((1, self.output_size))  # Adding bias to the second layer of weights
    
    #Training the model - takes the training input and output, and batch value
    def train(self, training_inputs, training_outputs, batch_size):
      
        print('Network training with a batch size of '+str(batch_size))
        print('-----')
        
        #intialization of previous gradients 
        prev_grad_w2 = 0
        prev_grad_w1 = 0
        # momentum value
        mom_factor = 0.55
        
        #Developing a progress bar to give a good graphic on the progress
        progress_bar = tqdm(range(self.epochs))
        for k in progress_bar:
            increment = 0
            avg_error = 0
            while increment < training_inputs.shape[0]:
                #making batches of the training data and their respective labels
                tx = training_inputs[increment:increment+batch_size]
                ty = training_outputs[increment:increment+batch_size]
                
                # forward propogation of data
                z1 = np.dot(tx, self.weight1)
                a1 = sigmoid(z1)
                z2 = np.dot(a1, self.weight2) 
                y = softmax(z2)
                
                #back-propogation of data and error calulations  
                dy2 = y*(1-y) #derivative of final output
                dy1 = a1*(1-a1) #derivative of hidden layer output
                output_error = np.subtract(ty,y) #error in output layer
                avg_error = np.mean(output_error)
                output_delta = output_error * dy2 #element wise mulitplication
                hidden_error = np.dot(output_delta, self.weight2.T)
                hidden_layer_delta = hidden_error * dy1 #element wise multiplication
                
                #momentum
                momentum_factor_w1 = mom_factor * prev_grad_w1
                momentum_factor_w2 = mom_factor * prev_grad_w2
                
                #backpropogation calculation for each of the weights and biases
                dW2 = np.dot(a1.T, output_delta) 
                db2 = np.sum(output_delta, axis = 0)
                dW1 = np.dot(tx.T, hidden_layer_delta)
                db1 = np.sum(hidden_layer_delta, axis = 0)

                                
                #update weights and bias
                self.weight2 += (1./batch_size * self.lr * dW2) + momentum_factor_w2
                self.weight1 +=  (1./batch_size * self.lr * dW1) + momentum_factor_w1
                self.bias1 +=  1./batch_size* self.lr * db1
                
                #save the graident to use in next batch iteration
                prev_grad_w1 = 1./batch_size * self.lr * dW1
                prev_grad_w2 = 1./batch_size * self.lr * dW2
                              
                #increment to access next set of inputs
                increment =increment + batch_size
            print("Error at this epoch :  ")
            print(output_error)
            
    # Used to test the neural network - accepts the x or input values of the test data
    def test(self, test_input):
        #forward feed
        z1 = np.dot(test_input, self.weight1) 
        a1 = sigmoid(z1)
        z2 = np.dot(a1, self.weight2) 
        y = softmax(z2)
        ytest = np.array(y)
        return ytest
    
    def prediction(self, test_input, test_output):
        #forward feed
        z1 = np.dot(test_input, self.weight1) 
        a1 = sigmoid(z1)
        z2 = np.dot(a1, self.weight2) 
        y = softmax(z2)
        a = np.zeros((4951, 1))
     
      # To see how accurate the neural network is compared to the correct values
        acc = 0.0
        y = np.array(y)
        test_output = np.array(test_output)
        for i in range(4951):
            
            if np.argmax(y[i]) == np.argmax(test_output[i]):
                acc += 1
                a[i] = np.argmax(y[i])
        print("Accuracy of this model on the test data is: ", acc / 4951 * 100, "%")
        return a


## - Training the model with approporiate parameters

In [7]:
if __name__ == '__main__':
    #Setting up the main program, with a selection of approporiate number of hidden layer nodes
    model = BPNN(800)
    #Training the model, with an appropriate batch size
    final_model = model.train(X_train,y_train,10)

  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

The designed network will have 800 number of nodes in the hidden layer
-----
Network training with a batch size of 10
-----


  3%|██▊                                                                                | 1/30 [00:30<14:43, 30.45s/it]

Error at this epoch :  
       1.000000000000000000e+00  0.000000000000000000e+00  0.000000000000000000e+00.1  0.000000000000000000e+00.2
15795                 -0.999766                 -0.000001                    -0.39305                    0.000007
23654                 -0.000234                  0.000001                    -0.60695                   -0.000007


  7%|█████▌                                                                             | 2/30 [01:01<14:14, 30.53s/it]

...

Error at this epoch :  
       1.000000000000000000e+00  0.000000000000000000e+00  0.000000000000000000e+00.1  0.000000000000000000e+00.2
15795             -1.000000e+00             -9.911988e-21                   -0.090838                0.000000e+00
23654             -1.148246e-22              0.000000e+00                   -0.909162               -1.465079e-26


 97%|███████████████████████████████████████████████████████████████████████████████▎  | 29/30 [14:46<00:30, 30.51s/it]

Error at this epoch :  
       1.000000000000000000e+00  0.000000000000000000e+00  0.000000000000000000e+00.1  0.000000000000000000e+00.2
15795             -1.000000e+00             -1.712824e-20                    -0.19558                0.000000e+00
23654             -5.780712e-23              0.000000e+00                    -0.80442               -3.824976e-27


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [15:16<00:00, 30.56s/it]

Error at this epoch :  
       1.000000000000000000e+00  0.000000000000000000e+00  0.000000000000000000e+00.1  0.000000000000000000e+00.2
15795             -1.000000e+00             -3.023983e-20                    -0.36854                0.000000e+00
23654             -3.441290e-23              0.000000e+00                    -0.63146               -1.045402e-27


## - Making predictions on the 'X_test' data using the trained model

In [8]:
#Making precitions on the trained model, to justify the accuracy of the model
model.prediction(X_test,y_test)

Accuracy of this model on the test data is:  92.36517875176732 %


array([[1.],
       [2.],
       [2.],
       ...,
       [1.],
       [3.],
       [1.]])